Setup and loading InData class from a file

In [39]:
from indata_pro_ops import InData
import sqlalchemy as db
from sqlalchemy import inspect
import pandas as pd

## Accesing testing .json file (from server) and preprocessing it

In [3]:
# Temporary for testing purposes, preload test1.json
test1 = InData("test1.json")
test1.extract()
print(test1.username)
print(test1.open[1])

Tester_manual
2023-10-26 12:10:13


## Establishing connectrion with SQL server (MySQL localhost for testing purpose)

In [12]:
# Establishing connection, using Devart ODBC Driver (Devart ODBC Driver for MySQL)
conn = pyodbc.connect('Driver={Devart ODBC Driver for MySQL};'
                      'User ID=root;'
                      'Password=legolas112;'
                      'Server=127.0.0.1;'
                      'Database=smokeout;'
                      'Port=3306;'
                      'Trusted_Connection=yes;')
# To perform any SQL QUERYs in pyodbc cursor() is needed
cursor = conn.cursor()
# Building QUERY strings:
# -------------------------------------------
#               WARNING!!!
# In MySQL syntax is different than in other SQL applications so for testing
# and final implementation there is need for different coding
# -------------------------------------------

# Inserting username into `user` table
username_insert = "INSERT INTO `smokeout`.`user` (`username`) VALUES ('"+test1.username+"');"
print(username_insert)
cursor.execute(username_insert)

# Commiting changes
#conn.commit()

AttributeError: module 'sqlalchemy' has no attribute 'create_engie'

In [9]:
import mysql.connector as sql


db_connection = sql.connect(host='127.0.0.1', database='smokeout', 
user='root', password='legolas112')

df = pd.read_sql('SELECT * FROM table_name', con=db_connection)

C:\Users\01122464\AppData\Local\Temp\ipykernel_14884\2619999710.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM table_name', con=db_connection)


DatabaseError: Execution failed on sql 'SELECT * FROM table_name': 1146 (42S02): Table 'smokeout.table_name' doesn't exist

In [7]:
# Getting UserID from username - https://stackoverflow.com/questions/64565904/how-to-use-pd-read-sql-using-mysql-connector
UserID = pd.read_sql("SELECT UserID FROM use WHERE username="+test1.username, conn)
# Inserting opening and switching times
for i in test1.switch:
    switch_insert = "INSERT INTO `smokeout`.`user` (`username`) VALUES (`"+test1.username+"`);"

C:\Users\01122464\AppData\Local\Temp\ipykernel_14884\2596656094.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  UserID = pd.read_sql("SELECT UserID FROM use WHERE username="+test1.username, conn)


DatabaseError: Execution failed on sql 'SELECT UserID FROM use WHERE username=Tester_manual': ('HY000', "[HY000] [Devart][ODBC][MySQL]#42000You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'use WHERE username=Tester_manual' at line 1 (0) (SQLExecDirectW)")

SQLAlchemy approach

In [43]:
# In SQLAlchemy (SQL module to work with pandas) to start connection
# first step is to create engine:
# -----------------------------------------------------------------
# For that although, the connection string is needed
conn_string = ''
# To connect to local database we need: user, password, port and name of database:
PWD = 'legolas112'
USR = 'root'
PORT = '3306'
DB = 'smokeout'
SQLALCHEMY_DATABASE_URI = 'mysql://{}:{}@localhost:{}/{}'.format(USR, PWD, PORT, DB)
conn_string = SQLALCHEMY_DATABASE_URI

engine = db.create_engine(conn_string)
conn = engine.connect()

# Extracting the metadata (needed to access the tables)
metadata = db.MetaData()
# Accessing respective tables: user and inapp_user_data
user = db.Table('user', metadata, autoload=True, autoload_with=engine)
inapp_user_data = db.Table('inapp_user_data', metadata, autoload=True, autoload_with=engine)
# printing that + also just columns
print(repr(metadata.tables['user']))
print(repr(metadata.tables['inapp_user_data']))
print(user.columns.keys())
# =====================================================================


# =====================================================================
# Establishing UserID, with known username <from .json>
# Select column `UserID` from table `user` where column `username` is equal to test1.username
out = conn.execute(db.select(user.c.UserID).where(user.c.username == test1.username))
UserID = out.fetchall()
# The format of this data is not friendly:
print(UserID)
for i in UserID[0]:
    UserID = i


Table('user', MetaData(), Column('UserID', INTEGER(), table=<user>, primary_key=True, nullable=False), Column('username', VARCHAR(collation='utf8mb4_polish_ci', length=30), table=<user>), Column('StartTime', DATETIME(), table=<user>), Column('PlanID', INTEGER(), ForeignKey('usage_plans.PlanID'), table=<user>), schema=None)
Table('inapp_user_data', MetaData(), Column('InDataID', INTEGER(), table=<inapp_user_data>, primary_key=True, nullable=False), Column('UserID', INTEGER(), ForeignKey('user.UserID'), table=<inapp_user_data>), Column('OpenPckg', DATETIME(), table=<inapp_user_data>), Column('SwitchPckg', DATETIME(), table=<inapp_user_data>), schema=None)
['UserID', 'username', 'StartTime', 'PlanID']
[(7,)]
